<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span> NLP : Entrainnement et sauvegarde du modèle </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

* [Installations](#section-1)
* [Imports](#section-2)
* [Choix des paramètres](#section-3)
* [Lecture des données](#section-4)
* [Preprocessing](#section-5)
* [Création du modèle](#section-6)
* [Entrainement du modèle](#section-7)
* [Prédiction des données](#section-8)
* [Sauvegarde du modèle](#section-9)
* [Test du modèle](#section-10)

<a id="section-1"></a>
# <span>1. Installation des packages</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

!pip3 install transformers
!pip3 install datasets
!pip install sentencepiece

Tue Dec 27 13:49:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# <span>2. Imports </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [2]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import metrics

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

# torch 
import torch
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# nltk 
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<a id="section-3"></a>
# <span>3. Choix des paramètres</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

<p style="text-align:justify;padding:20px;"> Dans cette section, nous pouvons paramétriser notre modèle. Pour cela, on disposible de plusieurs paramètres. Parmis ces paramètres, on retrouve le choix de fenètre, le batchSize, le pas d'apprentissage et le nombre d'épochs. </p>

In [4]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 5
LEARNING_RATE = 1e-05

LEN_TEXT = 200
OVERLAP = 50

DONNEE_AJOUTEES = 500

TRANSFORMER_NAME = "cmarkea/distilcamembert-base"
TRAIN_SIZE = 0.8

<a id="section-4"></a>
# <span>4. Lecture des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [5]:
train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
added_path = '/content/drive/MyDrive/HandOnAI_2_NLP/added_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

df = pd.read_csv(train_path)
df_added = pd.read_csv(added_path)
df_test = pd.read_csv(test_path)

# suppression des colonnes inutiles 
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_added.rename(columns = {'french':'data'}, inplace = True)
df_added = df_added.drop(['Unnamed: 0'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

df = df.append(df_added[:DONNEE_AJOUTEES], ignore_index=True)

<a id="section-5"></a>
# <span>5. Preprocessing</span>
<hr style="border-bottom: solid;background-color:light;color:black;">
<p style="text-align:justify;padding:20px;"> La partie preprocessing peut etre scindée en plusieurs étapes. En premier, nous allons appliquer un nettoyage de données. Suite à ce nettoyage, les données seront convertis dans le format adéquat. </p>



## <span>5.1 Nettoyage de données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

1.   Suppression des stopwords : les stopwords sont des mots qui n'apportent pas d'informations supplémentaires au texte. Afin de réduire la taille des news, on pourrait éffectuer la suppression de ceux-ci.
2.   Suppression des caractères spéciaux : en regardant plusieurs news, nous avons constaté qu'il y avait plusieurs caractères spéciaux. N'apportant aucune information supplémentaires, nous pouvons les supprimer. 

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    #text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [7]:
df['data'] = df['data'].apply(clean_text)
df_test['data'] = df_test['data'].apply(clean_text)

df = df.drop(df.index[1430])
df = df.drop(df.index[1429])
df = df.drop(df.index[1180])
df = df.drop(df.index[1136])
df = df.reset_index()

## <span>5.2 Découpage des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [8]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//(LEN_TEXT - OVERLAP) >0:
        n = len(text1.split())//(LEN_TEXT - OVERLAP)
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:LEN_TEXT]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*(LEN_TEXT - OVERLAP):w*(LEN_TEXT - OVERLAP) + LEN_TEXT]
            l_total.append(" ".join(l_parcial))
    return l_total

In [9]:
df['text_split'] = df['data'].apply(get_split)
df['len_split'] = df['text_split'].apply(lambda x: len(x))

df_test['text_split'] = df_test['data'].apply(get_split)
df_test['len_split'] = df_test['text_split'].apply(lambda x: len(x))

In [10]:
for index, row in df.iterrows():
  if len(row['text_split']) > 1: 
    print(index)
    break

print(df['text_split'][31][0].split()[(LEN_TEXT - OVERLAP):])
print(df['text_split'][31][1].split()[:OVERLAP])

3
['effet', 'domino', 'fait', 'chaîne', 'alimentaire', '.', 'refroidissement', 'suivi', 'ensuite', "d'un", 'emballement', 'températures', 'effet', 'serre', '.', 'cette', 'théorie', 'a', 'confirmée', 'début', 'années', '1990', 'lorsque', 'a', 'finalement', 'identifié', 'cratère', "d'impact", 'situé', 'chicxulub', 'péninsule', 'yucatan', 'mexique.', 'toutefois', "s'accorde", "aujourd'hui", 'reconnaître', 'certaine', 'pertinence', 'théorie', 'concurrente', 'avancée', 'géophysicien', 'français', 'vincent', 'courtillot', 'collègues.', 'fait', 'intervenir', 'éruptions']
['effet', 'domino', 'fait', 'chaîne', 'alimentaire', '.', 'refroidissement', 'suivi', 'ensuite', "d'un", 'emballement', 'températures', 'effet', 'serre', '.', 'cette', 'théorie', 'a', 'confirmée', 'début', 'années', '1990', 'lorsque', 'a', 'finalement', 'identifié', 'cratère', "d'impact", 'situé', 'chicxulub', 'péninsule', 'yucatan', 'mexique.', 'toutefois', "s'accorde", "aujourd'hui", 'reconnaître', 'certaine', 'pertinence',

## <span>5.3 Reformulation du labels</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [11]:
def create_df(df): 
  train_l = []
  label_l = []
  for idx,row in df.iterrows():
      for l in row['text_split']:
          train_l.append(l)
          label_l.append([1 if row['label'] == i else 0 for i in range(2)])

  return pd.DataFrame({'data':train_l, 'label':label_l})

In [12]:
cleaned_df = create_df(df)
cleaned_df_test = create_df(df_test)

## <span>5.4 Création du dataset</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, is_target = True):
        self.tokenizer = tokenizer
        self.df = dataframe
        self.text = dataframe.data
        self.max_len = max_len
        if is_target: 
          self.targets = self.df.label
        else: 
          self.targets = None

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        if self.targets is None: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
          }
        else: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
              'targets': torch.tensor(self.targets[index], dtype=torch.float)
          }

In [14]:
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_NAME)

train_dataset = cleaned_df.sample(frac=TRAIN_SIZE,random_state=200)
test_dataset = cleaned_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(cleaned_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(cleaned_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(cleaned_df_test, tokenizer, MAX_LEN)

Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/732 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

FULL Dataset: (3242, 2)
TRAIN Dataset: (2594, 2)
TEST Dataset: (648, 2)


## <span>5.5 Création du dataloader</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id="section-6"></a>
# <span>6. Création du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [16]:
class BERTClass(torch.nn.Module):
    def __init__(self):
      super(BERTClass, self).__init__()
      self.l1 = CamembertModel.from_pretrained(TRANSFORMER_NAME)
      self.l3 = torch.nn.Linear(768, 2) #2 = binary classification
    
    def forward(self, ids, mask, token_type_ids):
      output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
      output = self.l3(output_1['pooler_output'])

      return output

model = BERTClass()
model.to(device)

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream

BERTClass(
  (l1): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0): CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [17]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [18]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)
        targets = data['targets'].to(device)
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

<a id="section-7"></a>
# <span>7. Entrainement du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [19]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  0.6794394254684448
Epoch: 0, Loss:  0.6351193785667419
Epoch: 0, Loss:  0.5838422179222107
Epoch: 0, Loss:  0.40035152435302734
Epoch: 0, Loss:  0.4722016453742981
Epoch: 0, Loss:  0.7619262933731079
Epoch: 0, Loss:  0.39127400517463684
Epoch: 0, Loss:  0.5045604109764099
Epoch: 0, Loss:  0.45945045351982117
Epoch: 0, Loss:  0.1784716099500656
Epoch: 0, Loss:  0.5484887957572937
Epoch: 0, Loss:  0.345293790102005
Epoch: 0, Loss:  0.11309938877820969
Epoch: 0, Loss:  0.3322291672229767
Epoch: 0, Loss:  0.5409688949584961
Epoch: 0, Loss:  0.5929970145225525
Epoch: 0, Loss:  0.3839965760707855
Epoch: 0, Loss:  0.572950005531311
Epoch: 0, Loss:  0.18636520206928253
Epoch: 0, Loss:  0.4218113124370575
Epoch: 0, Loss:  0.35644543170928955
Epoch: 0, Loss:  0.2995882034301758
Epoch: 0, Loss:  0.2682546079158783
Epoch: 0, Loss:  0.2511405050754547
Epoch: 0, Loss:  0.07583804428577423
Epoch: 0, Loss:  0.2419230043888092
Epoch: 0, Loss:  0.1951153725385666
Epoch: 0, Loss:  0.2840

<a id="section-8"></a>
# <span>8. Prédiction des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [20]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
      for _, data in enumerate(testing_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        fin_targets.extend(targets.cpu().detach().numpy().tolist())

        m = torch.nn.Softmax(dim=1)
        fin_outputs.extend(torch.round(m(outputs)).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [21]:
outputs, targets = validation()
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.9696969696969697
F1 Score (Micro) = 0.9696969696969697
F1 Score (Macro) = 0.9595330005224357


In [22]:
cleaned_df_test['pred'] = outputs
cleaned_df_test.head()

,data,label,pred
0,président groupe lrem a pris toutes pincettes....,"[1, 0]","[1.0, 0.0]"
1,lrem l’introduction cette mesure d’âge a braqu...,"[1, 0]","[1.0, 0.0]"
2,villes françaises qualité l'air meilleure moin...,"[1, 0]","[1.0, 0.0]"
3,bordeaux lille égalité ensuite rennes puis tou...,"[1, 0]","[1.0, 0.0]"
4,cop25 vient s'achever laisse goût amer certain...,"[1, 0]","[1.0, 0.0]"


In [23]:
pos = 0
df_test['pred'] = [list() for x in range(len(df_test.index))]
for idx,row in df_test.iterrows():
  for i in range(row['len_split']): 
    row['pred'].append(cleaned_df_test.loc[pos]['pred'])
    pos += 1

In [25]:
df_test['prediction'] = df_test['pred'].apply(lambda x: [1, 0] if np.argmax(np.sum(x, axis = 0)) == 0 else [0, 1])
df_test['label_pred'] = df_test['pred'].apply(lambda x: np.argmax(np.sum(x, axis = 0)))

In [26]:
accuracy = metrics.accuracy_score(df_test['label_pred'], df_test['label'])
f1_score_micro = metrics.f1_score(df_test['label_pred'], df_test['label'], average='micro')
f1_score_macro = metrics.f1_score(df_test['label_pred'], df_test['label'], average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.9567901234567902
F1 Score (Micro) = 0.9567901234567902
F1 Score (Macro) = 0.9560031210797901


<a id="section-9"></a>
# <span>9. Sauvegarde du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [27]:
checkpoint = {'model': BERTClass(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, '/content/drive/MyDrive/HandOnAI_2_NLP/transformer_model.pth')

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream